In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import glob
import os
import math
from os import listdir
import data_visualization as dv
import pickle
import scipy 
from scipy.stats import norm
from scipy import stats
from tqdm import tqdm 

In [5]:
# Fields pull and pull list
experiment1 = 1
if experiment1==1:
    os.chdir('D:\Subject_Data\Seth_MatchPennies_Agent_Pilot_v1')
PATH = os.getcwd()
analysis_pull_list_react = []
analysis_pull_list_task = []
fields_pull = []
with open(PATH+"\\Analysis_Pull_List_React.txt","r") as pull_file:
    analysis_pull_list_react = pull_file.read().splitlines()
with open(PATH+"\\Analysis_Pull_List_Task.txt","r") as pull_file:
    analysis_pull_list_task = pull_file.read().splitlines()
with open(PATH+"\\Fields_Pull.txt","r") as fields_pull:
    fields_pull = fields_pull.read().splitlines()  

# SPECIFIC TO EXPERIMENT    
reaction_trials = 50
timing_trials = 50   
reaction_timing_trials = reaction_trials + timing_trials
num_trials = 80 #number of trials in each block 
num_blocks = 8
tot_trials = (num_trials*num_blocks)
data_columns = len(fields_pull)
trial_time = 1500
num_subjects = len(analysis_pull_list_react)

# Need to be kept outside subject for-loop
trial_end = np.zeros([num_subjects,num_blocks,num_trials,2])*np.nan 
trial_start = np.zeros((num_subjects,num_blocks,num_trials))*np.nan 
filename_storage = np.empty((num_subjects,num_blocks,num_trials), dtype = object)
csv_files = np.empty((num_subjects,tot_trials), dtype = object)  
csv_block_ordered = np.empty((num_subjects, num_blocks, num_trials))
reaction_timing_trial_table = np.empty((num_subjects, reaction_timing_trials, 4), int)
task_trial_table = np.empty((num_subjects, tot_trials, 4))

player_decision_array = np.empty((num_subjects,num_blocks,num_trials))     
player_reach_time = np.zeros([num_subjects,num_blocks,num_trials])*np.nan 
player_left_start_time = np.zeros([num_subjects,num_blocks,num_trials])*np.nan 

hand_pos_data = np.zeros((num_subjects, num_blocks, num_trials, trial_time+500, 4))*np.nan

event_codes = np.empty((num_subjects, num_blocks, num_trials, trial_time+500), str)
list_event_codes = ["E_SOUND_SIGNAL","E_END_REACHED_A", "E_END_REACHED_B"] 
tp_list = ("TP4","TP3","TP2")
scanned_files = []

win_check_R = np.zeros((num_subjects,num_blocks))
win_check_L = np.zeros((num_subjects,num_blocks))
num_miss_check_R = np.zeros((num_subjects,num_blocks))
num_miss_check_L = np.zeros((num_subjects,num_blocks))

rt_decision_array = np.empty((num_subjects,reaction_timing_trials))  
rmt_decision_array = np.empty((num_subjects,reaction_timing_trials))  
rt_trial_start = np.zeros((num_subjects,reaction_timing_trials))*np.nan 
reaction_movement_time = np.zeros([num_subjects,reaction_timing_trials])*np.nan 
reaction_time = np.zeros([num_subjects,reaction_trials])*np.nan 
timing_reach_time = np.zeros([num_subjects,timing_trials])*np.nan 
rt_hand_pos_data = np.zeros((num_subjects, reaction_timing_trials, trial_time+1000, 4))*np.nan

#########
agent_reach_time = np.zeros((num_subjects,num_blocks,num_trials))
agent_target_selection = np.zeros((num_subjects,num_blocks,num_trials))

## Target information for Right Hand
start1x = 0.55
start1y = 0.15
start1_diameter = 0.015
target1x = 0.65
target1y= 0.3
t1_pos = np.sqrt(target1x**2 + target1y**2)
target1_diameter = 0.04

target2x = 0.45
target2y = 0.3
t2_pos = np.sqrt(target2x**2 + target2y**2)
target2_diameter = 0.04

## for Left Hand
start2x = -0.55
start2y = 0.15
start2_diameter = 0.015
target3x = -0.45
target3y = 0.3
t3_pos = np.sqrt(target3x**2 + target3y**2)
target3_diameter = 0.04

target4x = -0.65
target4y = 0.3
t4_pos = np.sqrt(target4x**2 + target4y**2)
target4_diameter = 0.04

In [6]:
#%% Get trial table for reaction and timing
for w in range(len(analysis_pull_list_react)): #len(pull_list)):
    print(w)
    sub_name = analysis_pull_list_react[w]
    path1 = PATH+'\\'+sub_name
    # csv_files[w,:] = find_csv_filenames(path1, "Visual") #get csv names
    # csv_files = csv_files.astype(str)
    reaction_timing_trial_table = pd.read_csv(path1+f'\\{sub_name}Trial_Table.csv')

#%% Get trial table for task
for w in range(len(analysis_pull_list_task)): #len(pull_list)):
    print(w)
    sub_name = analysis_pull_list_task[w]
    path1 = PATH+'\\'+sub_name
    # csv_files[w,:] = find_csv_filenames(path1, "Visual") #get csv names
    # csv_files = csv_files.astype(str)
    df = pd.read_csv(path1+f'\\{sub_name}Trial_Table.csv')
    df = df.loc[df['TP_Row']!=1]
    tt = np.array(df)
    task_trial_table[w,:,:] = df[['Trial_Num','Block_Row','Block_Step','TP_Row']]

0


OSError: [Errno 22] Invalid argument: 'D:\\Subject_Data\\Seth_MatchPennies_Agent_Pilot_v1\\Sub4_React\t\\Sub4_React\tTrial_Table.csv'

In [55]:
reaction_timing_trial_table.head()

,Trial_Num,Block_Row,Block_Step,TP_Row,Starting bar Targ No,Ending bar Targ No,Condition type,Condition time,Trial feedback,Biased Score Param,Biased score side,Biased score,Show cum scores,Spring constant wall,Damping constant wall,Score show time,Reward display,Agent Mean Reach Time,Agent SD Reach Time,Repeat_Trials
0,1,1,1,1,1,2,0,5000,2,0,0,0,0,6000,5,1000,0,0,0,0
1,2,1,2,1,1,2,0,5000,2,0,0,0,0,6000,5,1000,0,0,0,0
2,3,1,3,1,1,2,0,5000,2,0,0,0,0,6000,5,1000,0,0,0,0
3,4,1,4,1,1,2,0,5000,2,0,0,0,0,6000,5,1000,0,0,0,0
4,5,1,5,1,1,2,0,5000,2,0,0,0,0,6000,5,1000,0,0,0,0


In [56]:
#%% Get reaction time data
for w in range(num_subjects):
    sub_name = analysis_pull_list_react[w]
    print(w, sub_name)
    block_number = 1
    tp_num = 1
    for i in tqdm((range(reaction_timing_trials))):
        block_number = reaction_timing_trial_table.iloc[i]['Block_Row']
        tp_num = reaction_timing_trial_table.iloc[i]['TP_Row']
        block_trial_num = reaction_timing_trial_table.iloc[i]['Block_Step']
        x = tp_num - 1
        c = block_trial_num - 1
        filename = PATH+f"\\{sub_name}\\Seth_MatchPennies_Agent_Pilot_v1_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"

        data = pd.read_csv(filename, low_memory=False)
        rt_trial_start[w,c]= int(data[data['Event_Codes']=='E_SOUND_SIGNAL'].index[0]) #if event codes column is equal to the list event codes, give the index (time)
        rt_trial_start = rt_trial_start.astype(int)
        end_time = rt_trial_start[w,c]+2500

        data = np.array(data)
        hand_data = data[:,0:4] # data is only columns 0 thru 3 (RHX,RHY,LHX,LHY)
        hand_data = hand_data.astype(float)
        rt_hand_pos_data[w,c,:,:] = hand_data[rt_trial_start[w,c]:end_time,:]
         # LEFT HAND
        lhx = rt_hand_pos_data[w,c,:,2]
        lhy = rt_hand_pos_data[w,c,:,3]
        if tp_num == 1:
            q = np.argwhere(np.sqrt((lhx-target3x)**2 + (lhy-target3y)**2) < target3_diameter/2)
            r = np.argwhere(np.sqrt((lhx-target4x)**2 + (lhy-target4y)**2) < target4_diameter/2) 
            s = np.argwhere(np.sqrt((lhx-start2x)**2 + (lhy-start2y)**2) > start2_diameter/2)
            if np.size(s)>0: 
                reaction_time[w,c] = s[0]
            if np.size(q)>0: #if LH enters right target
                rmt_decision_array[w,c] = 1
                reaction_movement_time[w,c] = q[0]
            elif np.size(r)>0: #if LH enters left target
                rmt_decision_array[w,c] = -1   
                reaction_movement_time[w,c] = r[0]
        if tp_num == 2:
            q = np.argwhere(np.sqrt((lhx-start2x)**2 + (lhy-target3y)**2) < target3_diameter/2) # THIS NEEDS TO BE THE CENTER TARGET... x poisition should be the start
            if np.size(q)>0:
                timing_reach_time[w,c] = q[0]


0 Sub1_React


100%|██████████| 100/100 [00:03<00:00, 29.89it/s]


1 Sub3_React


100%|██████████| 100/100 [00:03<00:00, 31.36it/s]


In [1]:
task_trial_table

NameError: name 'task_trial_table' is not defined

In [14]:
#%% Extract Hand Position data
for w in range(num_subjects): #len(pull_list)):
    print(w)
    sub_name = analysis_pull_list_task[w]
    print(sub_name)
    block_number = task_trial_table [w,:,1]
    block_trial_num = task_trial_table[w,:,2]
    tp_num = task_trial_table[w,:,3] #allows the same TP order every time
    for i in tqdm((range(tot_trials))):
        x = tp_num[i] - 3 #used for indexing, subtraction number needs to change to 1 if I start with TP1 
        c = block_trial_num[i] - 1 #used for indexing, block trial number starts with 1
        # if tp_num[i] != tp_num[i-1] and i>0:
        #     print(x)
        
        filename = PATH+f"\\{sub_name}\\Seth_MatchPennies_Agent_Pilot_{sub_name}_C{block_number[i]}_TP{tp_num[i]}_T{block_trial_num[i]}.csv"
        filename_storage[w,x,c] = filename
        data = pd.read_csv(filename, low_memory=False) #read in data
        #Create start time and max end time array
        score_index = int(data[data['Event_Codes']=='E_END_TRIAL'].index[0]) 
        trial_start[w,x,c]= int(data[data['Event_Codes']==list_event_codes[0]].index[0]) #if event codes column is equal to the list event codes, give the index (time)
        trial_start = trial_start.astype(int)
        end_time = trial_start[w,x,c]+2000
        agent_target_select_index = int(data[data['Event_Codes']=='E_END_TRIAL'].index[0]) 
        # Get Agent mean reach time... data starts at next index from E_SOUNG_SIGNAL aka trial_start
        agent_reach_time[w,x,c] = data.iloc[trial_start[w,x,c]+1]['Agent_Reach_Time']
        agent_target_selection[w,x,c] = data.iloc[agent_target_select_index]['Agent_Target_Selection']
        data = np.array(data)
        event_codes[w,x,c,:] = data[trial_start[w,x,c]:end_time,4] # create event codes array
        hand_data = data[:,0:4] # data is only columns 0 thru 3 (RHX,RHY,LHX,LHY)
        hand_data = hand_data.astype(float)
        hand_pos_data[w,x,c,:,:] = hand_data[trial_start[w,x,c]:end_time,:]
        
        # If you want the scores
        # scoreA = data[score_index, 4]
        # scoreB = data[score_index, 5]
        # if scoreA == 1:
        #     win_check_R[w,x]+=1
        # if scoreB == 1:
        #     win_check_L[w,x]+=1
        # print(filename)
        # print(data[trial_start[w,x,c]:end_time,:])
        #------------------------------------------------------------------
        # # RIGHT HAND
        # rhx1 = data[trial_start[w,x,c]:end_time,0]
        # rhy1 = data[trial_start[w,x,c]:end_time,1]
        # rhx = hand_pos_data[w,x,c,:,0]
        # rhy = hand_pos_data[w,x,c,:,1]
        # if np.any(rhx1 != rhx) or np.any(rhy1 !=rhy):
        #     print("fail")
        # z = np.argwhere(np.sqrt((rhx-target1x)**2 + (rhy-target1y)**2) < target1_diameter/2)
        # t = np.argwhere(np.sqrt((rhx-target2x)**2 + (rhy-target2y)**2) < target2_diameter/2)
        # if np.size(z)>0: #if RH enters right target
        #     player_decision_array[w,x,c] = 1
        #     target_reached_R[w,x,c] = z[0]
        
        # elif np.size(t)>0: #if RH enters left target
        #     player_decision_array[w,x,c] = -1
        #     target_reached_R[w,x,c] = t[0]
        # else: #No decision
        #     player_decision_array[w,x,c] = 0
        #     num_miss[w,x,0] = num_miss[w,x,0] + 1
           
        # LEFT HAND
        lhx = hand_pos_data[w,x,c,:,2]
        lhy = hand_pos_data[w,x,c,:,3]
        q = np.argwhere(np.sqrt((lhx-target3x)**2 + (lhy-target3y)**2) < target3_diameter/2)
        r = np.argwhere(np.sqrt((lhx-target4x)**2 + (lhy-target4y)**2) < target4_diameter/2) 
        s = np.argwhere(np.sqrt((lhx-start2x)**2 + (lhy-start2y)**2) > start2_diameter/2)
        if np.size(s)>0: 
            player_left_start_time[w,x,c] = s[0]
        if np.size(q)>0: #if LH enters right target
            player_decision_array[w,x,c] = 1
            player_reach_time[w,x,c] = q[0]
        elif np.size(r)>0: #if LH enters left target
            player_decision_array[w,x,c] = -1   
            player_reach_time[w,x,c] = r[0]
            
        if player_reach_time[w,x,c] > 1500 or np.isnan(player_reach_time[w,x,c]):
            player_decision_array[w,x,c] = 0
        # if w == 0 and x == 2 and c ==95:   
        #     lhx_new = hand_pos_data[w,x,c,:1500,2]
        #     lhy_new = hand_pos_data[w,x,c,:1500,3]
        #     #plot path for TargetNotReached
        #     #plt.figure(dpi=300)
        #     circleR = plt.Circle((target1x,target1y), target1_diameter/2, color = 'r', fill = False)
        #     circleL = plt.Circle((target2x,target2y), target2_diameter/2, color = 'r', fill = False)
        #     fig, ax = plt.subplots()
        #     ax.add_patch(circleR)
        #     ax.add_patch(circleL)
        #     plt.plot(lhx_new*-1,lhy_new) 
            
        #     plt.title("w =%1.0f " %w + "x=%1.0f "%x + "c=%1.0f"%c)
        #     plt.show()

0
Sub1


100%|██████████| 600/600 [00:16<00:00, 35.97it/s]


In [16]:
# Win, Loss, Indecision counts
# Make sure agent_target_selection replaces 2 with -1 from the first pilot
agent_target_selection = np.where(agent_target_selection ==2,-1,agent_target_selection)

player_indecisions = np.zeros((num_subjects, num_blocks))
player_wins = np.zeros((num_subjects, num_blocks))
player_incorrect_decisions = np.zeros((num_subjects,num_blocks))
for i in range(num_subjects):
    for j in range(num_blocks):
        player_indecisions[i,j] = np.count_nonzero(player_decision_array[i,j,:] == 0)
        player_wins[i,j] = np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == 1 , agent_target_selection[i,j,:] == 1))
        player_wins[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == -1 , agent_target_selection[i,j,:] == -1))
        player_wins[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == -1 , agent_target_selection[i,j,:] == 0))
        player_wins[i,j]+= np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == 1 , agent_target_selection[i,j,:] == 0))
        player_incorrect_decisions[i,j] = np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == 1 , agent_target_selection[i,j,:] == -1))
        player_incorrect_decisions[i,j] += np.count_nonzero(np.logical_and(player_decision_array[i,j,:] == -1 , agent_target_selection[i,j,:] == 1))
        player_indecisions[i,j] = np.count_nonzero(player_decision_array[i,j,:] == 0)
perc_player_indecisions = (player_indecisions / num_trials)*100
perc_player_wins = (player_wins / num_trials)*100

In [17]:
# Pickle
i=0
for subname in pull_list:
    data_path = PATH+f'\\Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(subname)
    # Reaction time 
    pickle.dump(rt_trial_start[i,:], open(data_path + f'{subname}_rt_trial_start.pkl', 'wb'))
    pickle.dump(reaction_movement_time[i,:], open(data_path + f'{subname}_reaction_movement_time.pkl', 'wb'))
    pickle.dump(reaction_time[i,:], open(data_path + f'{subname}_reaction_time.pkl', 'wb'))
    pickle.dump(rmt_decision_array[i,:], open(data_path + f'{subname}_rmt_decision_array.pkl', 'wb'))
    pickle.dump(player_indecisions[i,:],open(data_path + f'{subname}_player_indecisions.pkl', 'wb')) #
    pickle.dump(player_wins[i,:],open(data_path + f'{subname}_player_wins.pkl', 'wb'))
    pickle.dump(perc_player_indecisions[i,:],open(data_path + f'{subname}_perc_player_indecisions.pkl', 'wb')) #
    pickle.dump(perc_player_wins[i,:],open(data_path + f'{subname}_perc_player_wins.pkl', 'wb'))
    pickle.dump(hand_pos_data[i,:,:,:,:], open(data_path + f'{subname}_hand_pos_data.pkl', 'wb'))
    pickle.dump(trial_start[i,:,:], open(data_path + f'{subname}_trial_start.pkl', 'wb'))
    pickle.dump(player_reach_time[i,:,:], open(data_path + f'{subname}_player_reach_time.pkl', 'wb'))
    pickle.dump(trial_table[i,:,:], open(data_path + f'{subname}_trial_table.pkl', 'wb'))
    pickle.dump(player_decision_array[i,:,:], open(data_path + f'{subname}_player_decision_array.pkl', 'wb'))
    pickle.dump(player_incorrect_decisions[i,:], open(data_path + f'{subname}_player_incorrect_decisions.pkl', 'wb'))
    pickle.dump(player_left_start_time[i,:,:], open(data_path + f'{subname}_player_left_start_time.pkl', 'wb'))
    pickle.dump(player_decision_array[i,:,:], open(data_path + f'{subname}_player_decision_array.pkl', 'wb'))
    
    # Agent stuff
    pickle.dump(agent_target_selection[i,:,:],open(data_path + f'{subname}_agent_target_selection.pkl', 'wb'))
    pickle.dump(agent_reach_time[i,:,:],open(data_path + f'{subname}_agent_reach_time.pkl', 'wb'))

    i += 1         

Sub1
